In [1]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [2]:
TICKER_OPTIONS = {
    '1': ('Apple', 'AAPL'),
    '2': ('Microsoft', 'MSFT'),
    '3': ('Google', 'GOOGL'),
    '4': ('Amazon', 'AMZN'),
    '5': ('NVIDIA', 'NVDA'),
    '6': ('Tesla', 'TSLA'),
    '7': ('Meta (Facebook)', 'META'),
    '8': ('Berkshire Hathaway', 'BRK-B'),
    '9': ('Reliance Industries', 'RELIANCE.NS'),
    '10': ('Tata Consultancy (TCS)', 'TCS.NS'),
    '11': ('HDFC Bank', 'HDFCBANK.NS'),
    '12': ('Infosys', 'INFY.NS'),
    '13': ('ICICI Bank', 'ICICIBANK.NS'),
    '14': ('State Bank of India', 'SBIN.NS'),
    '15': ('Larsen & Toubro', 'LT.NS'),
    '16': ('ITC Limited', 'ITC.NS'),
    '0': ('Other', 'MANUAL_INPUT') # Special option for manual entry
}

In [ ]:
print("Please choose a stock from the options below:")
for key, (name, ticker) in TICKER_OPTIONS.items():
    if key != '0':
        print(f"  {key}: {name}")
print(f"  {key}: {TICKER_OPTIONS['0'][0]} (Enter any ticker manually)")

choice = ""
stock_ticker = ""
while choice not in TICKER_OPTIONS:
    choice = input("Enter your choice number: ").strip()
    if choice not in TICKER_OPTIONS:
        print("Invalid choice. Please select a number from the list.")

Please choose a stock from the options below:
  1: Apple
  2: Microsoft
  3: Google
  4: Amazon
  5: NVIDIA
  6: Tesla
  7: Meta (Facebook)
  8: Berkshire Hathaway
  9: Reliance Industries
  10: Tata Consultancy (TCS)
  11: HDFC Bank
  12: Infosys
  13: ICICI Bank
  14: State Bank of India
  15: Larsen & Toubro
  16: ITC Limited
  0: Other (Enter any ticker manually)


In [ ]:
chosen_name, chosen_ticker = TICKER_OPTIONS[choice]

if chosen_ticker == 'MANUAL_INPUT':
    print("\nExamples for custom tickers:")
    print("US Market: JPM, V | Indian Market: WIPRO.NS, KOTAKBANK.NS")
    stock_ticker = input("Please enter the custom stock ticker symbol: ").upper()
else:
    stock_ticker = chosen_ticker

In [ ]:
YEAR_OPTIONS = ['2025', '2026', '2027']
print("\nPlease choose the year for this model run:")
print(", ".join(YEAR_OPTIONS))

chosen_year = ""
while chosen_year not in YEAR_OPTIONS:
    chosen_year = input("Enter the year: ").strip()
    if chosen_year not in YEAR_OPTIONS:
        print("Invalid year. Please choose from the list.")

print(f"\nBuilding model for '{stock_ticker}' for the year {chosen_year}...")


In [ ]:
try:
    ticker_info = yf.Ticker(stock_ticker)
    stock_data = ticker_info.history(period="15y")
    
    # Get the company's full name, fall back to ticker if not found
    company_name = ticker_info.info.get('longName', stock_ticker)
    
    if stock_data.empty:
        print(f"No data found for ticker '{stock_ticker}'. Please check the symbol and try again.")
        exit()
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Could not retrieve data for '{stock_ticker}'. Please ensure the ticker is correct.")
    exit()

In [ ]:
df = stock_data[['Close']].copy()
df['Prev_Close'] = df['Close'].shift(1)
df.dropna(inplace=True)

print(f"\nData for '{company_name}' downloaded and prepared successfully.")
print(df.tail()) # Show the last 5 rows of data


In [ ]:
X = df[['Prev_Close']] # Feature
y = df['Close']      # Target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
print("\nModel training complete.")

In [ ]:
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f"Model Mean Squared Error (a measure of error): {mse:.2f}")

In [ ]:
last_known_price = df['Close'].iloc[-1]
# Create a DataFrame with the correct feature name to avoid the warning
prediction_input = pd.DataFrame([[last_known_price]], columns=['Prev_Close'])
next_day_prediction = model.predict(prediction_input)

In [ ]:
print("\n------------------ PREDICTION RESULT ------------------")
print(f"🏢 Company: {company_name} ({stock_ticker})")
print(f"🗓️ Last Known Price: ${last_known_price:.2f}")
print(f"🎯 Predicted Next Day Price: ${next_day_prediction[0]:.2f}")
print("-------------------------------------------------------")

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(y_train.index, y_train, color='blue', label='Training Data (Historical)')
plt.plot(y_test.index, y_test, color='green', label='Actual Price (Test Data)')
plt.plot(y_test.index, predictions, color='red', linestyle='--', label='Predicted Price')

plt.title(f'Stock Price Prediction for {company_name} ({stock_ticker})', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()

In [ ]:
graph_filename = 'stock_prediction_graph.png'
plt.savefig(graph_filename)
print(f"\n📈 A graph of the prediction has been saved as '{graph_filename}'")